In [1]:
"""
Panel-01
PLAN to upload a fasta file (single or multiple dataset)
DEMULTIPLEX if multi-dataset
Place resulting fasta files in sub-directory 
UNIQUE each subsequent file
Assign taxonomy to each: go with the easiest first: RDP
Then put the sequences, taxonomy, project and dataset into the database
"""
import os,sys
import shutil
#https://pypi.python.org/pypi/pyfasta/
from pyfasta import Fasta
from bin.remote_db_connector import MySQL_Connect
from bin.parse_rdp import RDP_File_Parser

In [10]:
"""
Panel-02
"""


# INPUT FASTA FILE INFO:
# Input fasta file (relative path)
# options are 'single' or 'multi' -dataset
file_info = {"name":'./test25.fa',"type":"single","sep":"|"}
#file_info = {"name":'./multi_ds.fa',"type":"multi","sep":" "}

# PROJECT INFORMATION
prj_info = {"name":"t18",  # REQUIRED 
            "public":1,       # 1=public; 0=private --Defaults to 1
            "owner":"admin",   # REQUIRED Owner must have vamps_user_name 
            "description":"testing desc",
            "title":"testing title",
            "funding":"01234"
           }
dataset_name = 'ds1'             # ONLY USED IF file_info_type = 'single'
taxonomy_assigner = 'rdp'        # also to include 'gast', 'rdp', 'spingo'
verbose = False                  # True or False: controls ammount of info output
base_dir = os.getcwd()
work_dir = os.path.join(base_dir,'work') # sub-directory where files will be (create this by hand!)
path_to_rdp_classifier = ''
path_to_gast = ''
path_to_spingo = ''

# MySQL DB CONNECTION INFORMATION
db_info = {"host":"localhost","name":"vamps_development","user":"ruby","pword":"ruby"}


In [11]:
"""
Panel-03
VALIDATE DATABASE CONNECTION
CHECK PROJECT NAME
CHECK VAMPS USERNAME
"""
conn_success = True
print('Testing Database Connection:')
try:
    db = MySQL_Connect(HOST=db_info["host"], DB=db_info["name"], USER=db_info["user"], PWORD=db_info["pword"])
    q = "SELECT * FROM project limit 2"
    db.cur.execute(q)
    result = db.cur.fetchall()
    for item in result:
        print(item)
    print('Done; Connection Success')
except:
    sys.exit('CONNECTION FAILURE --Exiting')
    conn_success = False

if conn_success:
    """ PROJECT """
    q = "SELECT project_id FROM project WHERE project='"+prj_info['name']+"'"    
    db.cur.execute(q)
    numrows = int(db.cur.rowcount)
    if numrows > 0:
        from random import randint
        print('Project already exists: '+prj_info['name'])
        prj_info['name'] = prj_info['name']+'_'+str(randint(100,999))
        print('Changing project name to:',prj_info['name'])
    else:
        print('Validated project name:\t\t',prj_info['name'])
    
    """ OWNER """
    q = "SELECT user_id FROM user WHERE username='"+prj_info['owner']+"'"
    db.cur.execute(q)
    numrows = int(db.cur.rowcount)
    if numrows==0:
        sys.exit('COULD NOT FIND VAMPS USER:',prj_info['owner']+' --Exiting')
    else:
        row = db.cur.fetchone()
        prj_info['owner_id'] = row[0]
        print('Validated VAMPS username:\t',prj_info['owner'],'(userID:'+str(prj_info['owner_id'])+')')

    

Testing Database Connection:
(44, 'ICM_SPO_Ev9', 'Title', 'Project Descriptionx', '6vB_OPS_MCI', 'myfunding', 48, 1)
(46, 'ICM_LCY_Bv6', 'Title', 'Project Descriptions', '6vB_YCL_MCI', 'myfunding', 48, 1)
Done; Connection Success
Project already exists: t18
Changing project name to: t18_637
Validated VAMPS username:	 admin (userID:48)


In [16]:
"""
Panel-04
CONFIRM/IDENTIFY FASTA TYPE: single or multi
Examine deflines: is dataset name first (multi) or sequenceID first (single)? 
Alter variable to suit: file_info.type (see Panel-02 above)
Alter variable to suit: file_info.sep (see above)
Also get total_seq_count here
"""
print('Running....')
print_count = 5  # show only this many
print('First',str(print_count),'complete deflines')
print("Your selection (see Panel-02 above)--> file_type: '"+file_info["type"]+ "'; separator: '"+file_info["sep"]+"'")
print('Examine these for file type and separator (Change above if needed):')
total_seq_count = 0
f = Fasta(file_info["name"])
avg_seq_length = 0
for defline in f:
    if total_seq_count < print_count:
        print(' ',defline)
    avg_seq_length += len(f[defline]) 
    total_seq_count += 1
avg_seq_length = avg_seq_length/total_seq_count
print('avg_seq_length:',avg_seq_length,'bp')

Running....
First 5 complete deflines
Your selection (see Panel-02 above)--> file_type: 'single'; separator: '|'
Examine these for file type and separator (Change above if needed):
  FX18VMI01C550A|rank=0000078|x=1183.0|y=1112.0|length=95
  FX18VMI01COJBD|rank=0000195|x=982.0|y=1847.0|length=104
  FX18VMI01BZB8B|rank=0000074|x=695.0|y=1817.0|length=104
  FX18VMI01C5C71|rank=0000100|x=1174.0|y=671.0|length=106
  FX18VMI01A0USI|rank=0000201|x=302.0|y=2640.0|length=104
avg_seq_length: 100.93333333333334 bp


In [17]:
"""
Panel-05
DEMULTIPLEX (If multi dataset) Files.
"""
print('Running....')
dataset_lookup = {}     # dataset_lookup[ds] = ds_count
sequence_collector = {} # sequences[ds][seqid] = seq
n=0
for defline in f:
    # create new fasta here for mothur to unique, names
    #print(defline)
    items = defline.split(file_info["sep"])
    
    seq = f[defline]
    #print(seq)
  
    if file_info["type"] == 'multi':
        
        seqid = items[1].replace(':','_')
        ds = items[0].split('_')[0]
        if ds in dataset_lookup:
            dataset_lookup[ds] += 1
        else:
            dataset_lookup[ds] = 1
        if ds in sequence_collector:
            sequence_collector[ds][seqid] = seq
        else:
            sequence_collector[ds] = {}
            sequence_collector[ds][seqid] = seq
        
    else:  # single dataset format
        #print("single ds")
        dataset_lookup[dataset_name] = total_seq_count
        seqid = items[0]
        ds = dataset_name
        if dataset_name in sequence_collector:
            sequence_collector[ds][seqid] = seq
        else:
            sequence_collector[ds] = {}
            sequence_collector[ds][seqid] = seq
    if n == 0:
        print('Printing a single seqID and dataset name:')
        print('Do they look properly formated? -- if not go back and adjust the splits above.')
        print('representative seqid:\t',seqid)
        print('representative dataset:\t',ds)
        
    n += 1
if verbose:
    print(dataset_lookup)
#print(sequence_collector)
print("There are",len(dataset_lookup), 'datasets and',total_seq_count,'total sequences')
print('Done')
            

Running....
Printing a single seqID and dataset name:
Do they look properly formated? -- if not go back and adjust the splits above.
representative seqid:	 FX18VMI01C550A
representative dataset:	 ds1
There are 1 datasets and 30 total sequences
Done


In [18]:
"""
Panel-06
CREATE CLEAN FASTA FILES
Output: seqfile.fa in each 'work/ds' directory
TODO: Output unique seqs and counts directly rather than use 'mothur'
"""
print('Running....')
# delete and re-create entire 'work' directory
if os.path.exists(work_dir):
    shutil.rmtree(work_dir) 
os.mkdir(work_dir)

# create directory structure
for ds in dataset_lookup:
    if not os.path.exists(os.path.join(work_dir,ds)):
        os.makedirs(os.path.join(work_dir,ds))
        
# write clean individual fasta files:    
for ds in sequence_collector:
    cleanfile = os.path.join(work_dir,ds,'seqfile.fa')
    f = open(cleanfile, 'w')
    for seqid in sequence_collector[ds]:
        seq = sequence_collector[ds][seqid][:]        
        f.write('>'+seqid+'\n'+seq+'\n')
    f.close()
print('Done')

Running....
Done


In [19]:
""" 
Panel-07
UNIQUE Each Sequence file(s)
# using mothur
# mothur_cmd = mothur \"#unique.seqs(fasta=$FASTA)\"
Output: uniques.fa and names file in each 'work/ds' directory
"""
print('Running....')
for ds in dataset_lookup:
    os.chdir(os.path.join(work_dir,ds))
    seqfile = 'seqfile.fa'
    out_pts = seqfile.split('.')
    uniques_file = '.'.join(out_pts[:-1])+'.unique.'+out_pts[-1]
    names_file = '.'.join(out_pts[:-1])+'.names'
    mothur_cmd = "mothur \"#unique.seqs(fasta="+seqfile+")\""
    os.system(mothur_cmd)
    os.rename(uniques_file,'uniques.fa')
    os.rename(names_file,'names')
    os.chdir(base_dir)
os.chdir(base_dir)
print('Done')

Running....
Done


In [20]:
"""
Panel-08
ASSIGN Taxonomy
Output: rdpfile file in each 'work/ds' directory
"""
print('Running....')
if taxonomy_assigner == 'rdp':
    # RDP: Choose gene from: 16srrna, fungallsu, fungalits_warcup, fungalits_unite
    rdp_gene = '16srrna' # for Euks choose either of: fungallsu, fungalits_warcup, fungalits_unite
    # See "java -Xmx4000M -jar ../rdp_classifier/dist/classifier.jar -h" for RDP help
    rdp_jar_path = os.path.join(base_dir,'..','rdp_classifier','dist','classifier.jar')

    for ds in dataset_lookup:
        uniques_file = os.path.join(work_dir,ds,'uniques.fa')
        rdp_outfile = os.path.join(work_dir,ds,'rdpfile')
        rdp_cmd = 'java -Xmx4000M -jar '+rdp_jar_path+" -g "+rdp_gene+" -o "+rdp_outfile+" -f fixrank "+ uniques_file
        #print(rdp_cmd)
        os.system(rdp_cmd)
    print('Done')
else:
    print('No useful tax assigner')


Running....
Done


In [21]:
"""
Panel-09
PARSE RDP FILE(S) and WRITE OUTPUT
Output: vamps_data.csv file in each 'work/ds' directory
"""
print('Running....')

for ds in dataset_lookup:
    rdp_file     = os.path.join(work_dir,ds,'rdpfile')
    uniques_file = os.path.join(work_dir,ds,'uniques.fa')
    names_file   = os.path.join(work_dir,ds,'names')
    vampsfile = os.path.join(work_dir,ds,'vamps_data.csv')
    obj = RDP_File_Parser(rdpFile=rdp_file, 
                    uniquesFile=uniques_file, 
                    namesFile=names_file, 
                    outFile=vampsfile,
                    min_boot=0.8)
    obj.parse_file()
    obj.write_file()
print('Done')

Running....
Done


In [22]:
"""
Panel-10
DATABASE Step-1: create RANK_COLLECTOR
"""
RANK_COLLECTOR         = {}
SEQ_COLLECTOR          = {}
#SUMMED_TAX_COLLECTOR   = {}
#TAX_ID_BY_RANKID_N_TAX = {}
#SILVA_IDS_BY_TAX       = {}
DATASET_ID_BY_NAME     = {}
ranks    = ['domain','phylum','klass','order','family','genus','species','strain']
rank_ids = ['domain_id','phylum_id','klass_id','order_id','family_id','genus_id','species_id','strain_id']
accepted_domains = ['bacteria','archaea','eukarya','fungi','organelle','unknown']
sqlranks = "','".join(ranks)       
q = "SELECT rank,rank_id FROM rank WHERE rank in ('%s')" % (sqlranks)
if verbose: print(q)
db.cur.execute(q)
rows = db.cur.fetchall()
for row in rows:
    RANK_COLLECTOR[row[0]] = row[1]
print('RANK_COLLECTOR:')
print(RANK_COLLECTOR)
print('Done')
    

RANK_COLLECTOR:
{'genus': 6, 'family': 5, 'domain': 1, 'strain': 8, 'order': 4, 'klass': 3, 'phylum': 2, 'species': 7}
Done


In [23]:
"""
Panel-11
DATABASE Step-2
Read VAMPS File(s) and Push Taxonomy
"""
print('Running....')
#import csv
#lines = csv.reader(open(file_info['name'],"r"), delimiter='\t')
for ds in dataset_lookup:
    vampsfile = os.path.join(work_dir,ds,'vamps_data.csv')
    tax_collector = {}
    if ds not in SEQ_COLLECTOR:
        SEQ_COLLECTOR[ds] = {}
    #if ds not in SUMMED_TAX_COLLECTOR:
    #    SUMMED_TAX_COLLECTOR[ds]={}
    with open(vampsfile) as f:
        for line in f:
            items = line.strip().split('\t')
            seqid       = items[0]
            seq         = items[1]
            tax_string  = items[2]
            boot_score  = items[3]
            rank        = items[4]
            seq_count   = items[5]
            if rank == 'class': rank = 'klass' 
            if rank == 'orderx': rank = 'order'
            SEQ_COLLECTOR[ds][seq] = {'dataset':ds,                                  
                                      'taxonomy':tax_string,    
                                      'rank':rank,     
                                      'seq_count':seq_count,   
                                      'boot':boot_score
                                     }    
            SEQ_COLLECTOR[ds][seq]['rank_id'] = RANK_COLLECTOR[rank]
            tax_items = tax_string.split(';')
#             sumtax = ''
#             for i in range(0,8):
#                 rank_id = RANK_COLLECTOR[ranks[i]]
#                 if len(tax_items) > i:
#                     taxitem = tax_items[i]
#                 else:
#                     taxitem = ranks[i]+'_NA'
#                 sumtax += taxitem+';'

#                 #print ranks[i],rank_id,taxitem,sumtax,seq_count
#                 if rank_id in SUMMED_TAX_COLLECTOR[ds]:
#                     if sumtax[:-1] in SUMMED_TAX_COLLECTOR[ds][rank_id]:
#                         SUMMED_TAX_COLLECTOR[ds][rank_id][sumtax[:-1]] += int(seq_count)
#                     else:
#                         SUMMED_TAX_COLLECTOR[ds][rank_id][sumtax[:-1]] = int(seq_count)
#                 else:
#                     SUMMED_TAX_COLLECTOR[ds][rank_id] = {}
#                     SUMMED_TAX_COLLECTOR[ds][rank_id][sumtax[:-1]] = int(seq_count)
                
                
            if tax_items[0].lower() in accepted_domains:
                ids_by_rank = []
                for i in range(0,8):
                    #print i,len(tax_items),tax_items[i]
                    rank_name = ranks[i]
                    rank_id = RANK_COLLECTOR[ranks[i]]

                    if len(tax_items) > i:
                        if ranks[i] == 'species':
                            t = tax_items[i].lower()
                        else:
                            t = tax_items[i].capitalize()

                        if tax_items[i].lower() != (rank_name+'_NA').lower():
                            name_found = False
                            if rank_name in tax_collector:
                                tax_collector[rank_name].append(t)
                            else:
                                tax_collector[rank_name] = [t]
                    else:
                        t = rank_name+'_NA'



                    q2 = "INSERT IGNORE INTO `"+rank_name+"` (`"+rank_name+"`) VALUES('"+t+"')"
                    if verbose: print(q2)
                    db.cur.execute(q2)
                    db.conn.commit()
                    tax_id = db.cur.lastrowid
                    if tax_id == 0:
                        q3 = "SELECT "+rank_name+"_id FROM `"+rank_name+"` WHERE `"+rank_name+"` = '"+t+"'"
                        if verbose: print( q3 )
                        db.cur.execute(q3)
                        db.conn.commit()
                        row = db.cur.fetchone()
                        tax_id=row[0]
                    ids_by_rank.append(str(tax_id))
                    
                    if verbose: print( 'rank_id,t,tax_id '+str(rank_id)+' - '+t+' - '+str(tax_id)  )
#                     if rank_id in TAX_ID_BY_RANKID_N_TAX:
#                         TAX_ID_BY_RANKID_N_TAX[rank_id][t] = tax_id
#                     else:
#                         TAX_ID_BY_RANKID_N_TAX[rank_id]={}
#                         TAX_ID_BY_RANKID_N_TAX[rank_id][t] = tax_id
                    #ids_by_rank.append('1')
                if verbose: print(ids_by_rank )
                q4 =  "INSERT IGNORE INTO silva_taxonomy ("+','.join(rank_ids)+",created_at)"
                q4 += " VALUES("+','.join(ids_by_rank)+",CURRENT_TIMESTAMP())"
                if verbose: print(q4)
                db.cur.execute(q4)
                db.conn.commit()
                silva_tax_id = db.cur.lastrowid
                if silva_tax_id == 0:
                    q5 = "SELECT silva_taxonomy_id FROM silva_taxonomy WHERE ("
                    vals = ''
                    for i in range(0,len(rank_ids)):
                        vals += ' '+rank_ids[i]+"="+ids_by_rank[i]+' AND'
                    q5 = q5 + vals[0:-3] + ')'
                    if verbose: print(q5)
                    db.cur.execute(q5)
                    db.conn.commit()
                    row = db.cur.fetchone()
                    silva_tax_id=row[0]

                #SILVA_IDS_BY_TAX[tax_string] = silva_tax_id
                SEQ_COLLECTOR[ds][seq]['silva_tax_id'] = silva_tax_id
                db.conn.commit()

print('Done')

Running....
Done


In [24]:
"""
Panel-12
DATABASE Step-3
Push Sequences
"""
print('Running....')
for ds in dataset_lookup:
    for seq in SEQ_COLLECTOR[ds]:
        q = "INSERT IGNORE INTO sequence (sequence_comp) VALUES (COMPRESS('"+seq+"'))"
        if verbose: print(q)
        db.cur.execute(q)
        db.conn.commit()
        seqid = db.cur.lastrowid
        if seqid == 0:
            q2 = "SELECT sequence_id FROM sequence WHERE sequence_comp = COMPRESS('"+seq+"')"
            if verbose: print('DUP SEQ FOUND')
            db.cur.execute(q2)
            db.conn.commit()
            row = db.cur.fetchone()
            seqid=row[0]
        SEQ_COLLECTOR[ds][seq]['sequence_id'] = seqid
        silva_tax_id = str(SEQ_COLLECTOR[ds][seq]['silva_tax_id'])
        distance = str(SEQ_COLLECTOR[ds][seq]['boot'])
        if verbose: print( ds+' - '+seq+' - '+str(silva_tax_id))
        rank_id = str(SEQ_COLLECTOR[ds][seq]['rank_id'])
        if verbose: print( rank_id)
        q = "INSERT IGNORE INTO silva_taxonomy_info_per_seq"
        q += " (sequence_id,silva_taxonomy_id,gast_distance,refssu_id,rank_id)"
        q += " VALUES ('"+str(seqid)+"','"+silva_tax_id+"','"+distance+"','0','"+rank_id+"')"
        if verbose: print(q)
        db.cur.execute(q)
        db.conn.commit()
        silva_tax_seq_id = db.cur.lastrowid
        if verbose: print('1: '+str(silva_tax_seq_id))
        if silva_tax_seq_id == 0:
            q3 = "SELECT silva_taxonomy_info_per_seq_id FROM silva_taxonomy_info_per_seq"
            q3 += " WHERE sequence_id = '"+str(seqid)+"'"
            if verbose: print('DUP silva_tax_seq')
            if verbose: print(q3)
            db.cur.execute(q3)
            db.conn.commit()
            row = db.cur.fetchone()
            if verbose: print(row)
            silva_tax_seq_id=row[0]
            if verbose: print('0: '+str(silva_tax_seq_id))

        q4 = "INSERT IGNORE INTO sequence_uniq_info (sequence_id, silva_taxonomy_info_per_seq_id)"
        q4 += " VALUES('"+str(seqid)+"','"+str(silva_tax_seq_id)+"')"
        if verbose: print(q4)
        db.cur.execute(q4)
        db.conn.commit()
        ## don't see that we need to save uniq_ids
    db.conn.commit()
    print('Done')

Running....
Done


In [25]:
"""
Panel-13
DATABASE Step-4
Push Project
"""
desc  = prj_info["description"]
title = prj_info["title"]
proj  = prj_info['name']
rev   = prj_info['name'][::-1]
fund  = prj_info["funding"]
oid   = prj_info['owner_id']
pub   = prj_info['public']
fields = ['project','title','project_description','rev_project_name','funding','owner_user_id','public']

q = "INSERT INTO project ("+(',').join(fields)+")"
q += " VALUES('%s','%s','%s','%s','%s','%s','%s')"
q = q % (proj, title, desc, rev, fund, oid, pub)
print(q)
db.cur.execute(q)
db.conn.commit()
prj_info['project_id'] = db.cur.lastrowid
print("NEW Project: "+prj_info['name']+" (PID = "+str(prj_info['project_id'])+')')
print("Done")

INSERT INTO project (project,title,project_description,rev_project_name,funding,owner_user_id,public) VALUES('t18_637','testing title','testing desc','736_81t','01234','48','1')
NEW Project: t18_637 (PID = 112)
Done


In [26]:
"""
Panel-14
DATABASE Step-5
Push Dataset(s)
"""
fields = ['dataset','dataset_description','project_id']
for ds in dataset_lookup:
    q = "INSERT INTO dataset ("+(',').join(fields)+")"
    q += " VALUES('%s','%s','%s')"
    #print ds,desc,CONFIG_ITEMS['env_source_id'],CONFIG_ITEMS['project_id']
    q4 = q % (ds, ds+'_description', prj_info['project_id'])
    if verbose: print(q4)
    db.cur.execute(q4)
    did = db.cur.lastrowid
    DATASET_ID_BY_NAME[ds] = str(did)
    db.conn.commit()
    print("DATASET_ID_BY_NAME:")
    print(DATASET_ID_BY_NAME)
    

DATASET_ID_BY_NAME:
{'ds1': '859'}


In [28]:
"""
Panel-15
DATABASE Step-6
Push PDR Seqs
"""
classifier_id = '5' # '5' for rdp; '2' for gast (on local) See classifier table
for ds in dataset_lookup: 
    did = DATASET_ID_BY_NAME[ds]
    for seq in SEQ_COLLECTOR[ds]:        
        seqid = SEQ_COLLECTOR[ds][seq]['sequence_id']
        count = SEQ_COLLECTOR[ds][seq]['seq_count']
        q = "INSERT INTO sequence_pdr_info (dataset_id, sequence_id, seq_count, classifier_id)"
        q += " VALUES ('"+str(did)+"','"+str(seqid)+"','"+str(count)+"','"+classifier_id+"')"
        if verbose: print(q)
        db.cur.execute(q)
    db.conn.commit()
print("Done")
# from bin.load_to_database import Load_CSV_Data_File
# for ds in dataset_lookup:
    
#     vampsfile = os.path.join(work_dir,ds,'vamps_data.csv')
#     obj = Load_CSV_Data_File(inFile=vampsfile,
#                              project=prj_info["name"],
#                              dataset=ds,
#                              user=prj_info["owner"],
#                              public=prj_info["public"],
#                              dbhost=db_info['host'])



Done


In [29]:
"""
Panel-16 
Metadata (Required)
TODO: Fix This::Assume here that all the datasets get the same required metadata
"""
collector = {}
# items without ids:
collector['collection_date'] = "2016-12-25"
collector['latitude']        = "41.525480"
collector['longitude']       = "-70.674854"

# items with ids:
req_md2 = [
     {"table":"term",               "name":"env_biome",          "value":"sea water"},         # term     
     {"table":"target_gene",        "name":"target_gene",        "value":"16s"},               # target_gene
     {"table":"dna_region",         "name":"dna_region" ,        "value":"v6"},                # dna_region
     {"table":"sequencing_platform","name":"sequencing_platform","value":"illumina"},          # sequencing_platform
     {"table":"domain",             "name":"domain",             "value":"bacteria"},          # domain
     {"table":"term",               "name":"geo_loc_name",       "value":"NWCS"},              # term
     {"table":"term",               "name":"env_feature",        "value":"unknown"},           # term
     {"table":"term",               "name":"env_matter",         "value":"water"},             # term
     {"table":"env_package",        "name":"env_package",        "value":"water-marine"},      # env_package
     {"table":"run_key",            "name":"adapter_sequence",   "value":"unknown"},           # run_key
     {"table":"illumina_index",     "name":"illumina_index",     "value":"unknown"},           # illumina_index
     {"table":"primer_suite",       "name":"primer_suite",       "value":"Bacterial v6 Suite"},# primer_suite
     {"table":"run",                "name":"run",                "value":"unknown"}            # run
]

# get the ids from the names/values
for i,q in enumerate(req_md2):
    if req_md2[i]["table"] == 'term':
        name = 'term_name'
        id = 'term_id'
    elif req_md2[i]["table"] == 'run_key':
        name = 'run_key'
        id = 'run_key_id'        
    else:
        name = req_md2[i]["name"]
        id = name+"_id"
    q = "SELECT "+id+", "+name
    q += " FROM "+req_md2[i]["table"] 
    q += " WHERE "+name+" = '"+req_md2[i]["value"]+"';"
    if verbose: print(q)
    db.cur.execute(q)
    rowcount = db.cur.rowcount
    if rowcount == 0:
        print('no vals for ',req_md2[i])
        q2 = q[:q.find('=')]+"='unknown'" # ALL the tables should have 'unknown'
        if verbose: print(q2)
        db.cur.execute(q2)
        row2 = db.cur.fetchone()
        id = row2[0]
    else:
        row1 = db.cur.fetchone()
        id = row1[0]
         
    collector[req_md2[i]["name"]+"_id"] = id
    
print(collector)
set_order = collector.keys()
set_values = []
for name in set_order:
    set_values.append(str(collector[name]))

for ds in dataset_lookup:
    did = DATASET_ID_BY_NAME[ds]
    q = "INSERT INTO required_metadata_info (dataset_id,"+','.join(set_order)+")"
    q += " VALUES('%s',"
    for n in range(len(set_order)):
        q += "'%s',"
    q = q[:-1]
    q += ")"
    q = q % tuple([did]+set_values)
    if verbose: print(q)
    try:
        db.cur.execute(q)
    except:
        print("INSERT Failed:\n",q)
db.conn.commit()
    

no vals for  {'table': 'term', 'value': 'NWCS', 'name': 'geo_loc_name'}
{'adapter_sequence_id': 1, 'latitude': '41.525480', 'illumina_index_id': 83, 'env_matter_id': 1280, 'dna_region_id': 12, 'geo_loc_name_id': 9999, 'longitude': '-70.674854', 'sequencing_platform_id': 2, 'domain_id': 140108, 'env_biome_id': 1375, 'env_feature_id': 9999, 'run_id': 5543, 'env_package_id': 22, 'collection_date': '2016-12-25', 'target_gene_id': 1, 'primer_suite_id': 23}
